In [1]:
%matplotlib inline

In [2]:
cd ..

C:\Projects\python\recommender


In [3]:
from functools import partial
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import torch as T
import torch.optim as optim

from utils import get_log_dir

In [4]:
from datasets import TorchKaggle
from models import FMLearner, TorchFM, TorchHrmFM, TorchPrmeFM, TorchTransFM
from models.fm_learner import simple_loss, trans_loss

In [5]:
DEVICE = T.cuda.current_device()
BATCH = 2000
SHUFFLE = True
WORKERS = 0
item_path = Path("./inputs/kaggle/item.csv")

In [6]:
db = TorchKaggle(data_path=item_path, user_min=4)
db

Raw dataframe shape (476244, 7)
After drop nan shape: (429988, 7)
Original comptition size: 292
Original competitor size: 140065
Filtered competiter size: 27449
Filtered dataframe shape: (284806, 7)


In [7]:
db.config_db(batch_size=BATCH,
             shuffle=SHUFFLE,
             num_workers=WORKERS,
             device=DEVICE)

In [8]:
feat_dim = db.feat_dim
NUM_DIM = 124
INIT_MEAN = 0.1

## Create Criterion

In [9]:
# regst setting
LINEAR_REG = 1
EMB_REG = 1
TRANS_REG = 1

In [10]:
simple_loss_callback = partial(simple_loss, LINEAR_REG, EMB_REG)
simple_loss_callback

functools.partial(<function simple_loss at 0x000001EDD53DBAE8>, 1, 1)

In [11]:
trans_loss_callback = partial(trans_loss, LINEAR_REG, EMB_REG, TRANS_REG)
trans_loss_callback

functools.partial(<function trans_loss at 0x000001EDD5432BF8>, 1, 1, 1)

## Train Model

### Hyper-parameter

In [12]:
feat_dim = db.feat_dim
NUM_DIM = 124
INIT_MEAN = 0.1

### Train FM Model

In [13]:
LEARNING_RATE = 0.001
DECAY_FREQ = 1000
DECAY_GAMMA = 1

In [14]:
fm_model = TorchFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
fm_model

TorchFM()

In [15]:
adam_opt = optim.Adam(fm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt, step_size=DECAY_FREQ, gamma=DECAY_GAMMA)

In [16]:
fm_learner = FMLearner(fm_model, adam_opt, schedular, db)
fm_learner

In [17]:
fm_learner.fit(epoch=8,
               loss_callback=simple_loss_callback,
               log_dir=get_log_dir(ds_type='kaggle', model_type='fm'))

  0%|                                                                                                                                                                 | 0/8 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 48414.34139250202
Epoch 0 step 1: training accuarcy: 0.06
Epoch 0 step 1: training loss: 48823.16708975167
Epoch 0 step 2: training accuarcy: 0.07200000000000001
Epoch 0 step 2: training loss: 47717.218700082536
Epoch 0 step 3: training accuarcy: 0.1015
Epoch 0 step 3: training loss: 47393.68143863562
Epoch 0 step 4: training accuarcy: 0.1155
Epoch 0 step 4: training loss: 48496.713193549214
Epoch 0 step 5: training accuarcy: 0.094
Epoch 0 step 5: training loss: 47173.28324278945
Epoch 0 step 6: training accuarcy: 0.124
Epoch 0 step 6: training loss: 46900.029386558475
Epoch 0 step 7: training accuarcy: 0.123
Epoch 0 step 7: training loss: 46350.61429493471
Epoch 0 step 8: training accuarcy: 0.1355
Epoch 0 step 8: training loss: 46138.23666630378
Epoch 0 step 9: training accuarcy: 0.137
Epoch 0 step 9: training loss: 45006.27565242712
Epoch 0 step 10: training accuarcy: 0.152
Epoch 0 step 10: training loss: 44627.071136788574
Epoch 0 step 11: tra

Epoch 0 step 87: training loss: 1076.587434787167
Epoch 0 step 88: training accuarcy: 0.995
Epoch 0 step 88: training loss: 1053.1617675210052
Epoch 0 step 89: training accuarcy: 0.9985
Epoch 0 step 89: training loss: 1031.8444001514476
Epoch 0 step 90: training accuarcy: 0.998
Epoch 0 step 90: training loss: 1014.1985229698965
Epoch 0 step 91: training accuarcy: 0.9955
Epoch 0 step 91: training loss: 993.5876761668814
Epoch 0 step 92: training accuarcy: 0.9955
Epoch 0 step 92: training loss: 975.1689707038748
Epoch 0 step 93: training accuarcy: 0.9965
Epoch 0 step 93: training loss: 957.6688646031182
Epoch 0 step 94: training accuarcy: 0.9975
Epoch 0 step 94: training loss: 936.7250568067739
Epoch 0 step 95: training accuarcy: 0.9985
Epoch 0 step 95: training loss: 920.0271281022422
Epoch 0 step 96: training accuarcy: 0.998
Epoch 0 step 96: training loss: 905.0953875605263
Epoch 0 step 97: training accuarcy: 0.998
Epoch 0 step 97: training loss: 891.669021012198
Epoch 0 step 98: train

 12%|███████████████████▏                                                                                                                                     | 1/8 [00:51<06:02, 51.81s/it]

Epoch: 1
Epoch 1 step 115: training loss: 672.0394434303187
Epoch 1 step 116: training accuarcy: 1.0
Epoch 1 step 116: training loss: 667.3819621922501
Epoch 1 step 117: training accuarcy: 0.9975
Epoch 1 step 117: training loss: 660.4615419638493
Epoch 1 step 118: training accuarcy: 0.998
Epoch 1 step 118: training loss: 645.4609685067533
Epoch 1 step 119: training accuarcy: 0.9995
Epoch 1 step 119: training loss: 637.636123822194
Epoch 1 step 120: training accuarcy: 1.0
Epoch 1 step 120: training loss: 629.5567146957729
Epoch 1 step 121: training accuarcy: 1.0
Epoch 1 step 121: training loss: 625.1310815217778
Epoch 1 step 122: training accuarcy: 0.999
Epoch 1 step 122: training loss: 620.3083580548066
Epoch 1 step 123: training accuarcy: 0.997
Epoch 1 step 123: training loss: 608.4111965165807
Epoch 1 step 124: training accuarcy: 0.9995
Epoch 1 step 124: training loss: 603.5191606763558
Epoch 1 step 125: training accuarcy: 0.9985
Epoch 1 step 125: training loss: 599.2072506815748
Epo

...

Epoch 6 step 777: training accuarcy: 0.999
Epoch 6 step 777: training loss: 35.170973126211344
Epoch 6 step 778: training accuarcy: 0.9995
Epoch 6 step 778: training loss: 36.43577128564611
Epoch 6 step 779: training accuarcy: 0.999
Epoch 6 step 779: training loss: 34.7258736339958
Epoch 6 step 780: training accuarcy: 0.9995
Epoch 6 step 780: training loss: 36.739514658577285
Epoch 6 step 781: training accuarcy: 0.9995
Epoch 6 step 781: training loss: 36.4532933782602
Epoch 6 step 782: training accuarcy: 0.9995
Epoch 6 step 782: training loss: 35.71718085836716
Epoch 6 step 783: training accuarcy: 1.0
Epoch 6 step 783: training loss: 35.66251654186964
Epoch 6 step 784: training accuarcy: 0.9995
Epoch 6 step 784: training loss: 33.32077106700435
Epoch 6 step 785: training accuarcy: 1.0
Epoch 6 step 785: training loss: 35.82718247917695
Epoch 6 step 786: training accuarcy: 1.0
Epoch 6 step 786: training loss: 34.89929089225908
Epoch 6 step 787: training accuarcy: 1.0
Epoch 6 step 787: tr

 88%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                   | 7/8 [05:44<00:49, 49.16s/it]

Epoch: 7
Epoch 7 step 805: training loss: 32.36311428430497
Epoch 7 step 806: training accuarcy: 1.0
Epoch 7 step 806: training loss: 33.652171504709585
Epoch 7 step 807: training accuarcy: 0.999
Epoch 7 step 807: training loss: 32.196356396777794
Epoch 7 step 808: training accuarcy: 1.0
Epoch 7 step 808: training loss: 33.07616727402335
Epoch 7 step 809: training accuarcy: 1.0
Epoch 7 step 809: training loss: 32.44762822017671
Epoch 7 step 810: training accuarcy: 0.9995
Epoch 7 step 810: training loss: 32.81408076595477
Epoch 7 step 811: training accuarcy: 1.0
Epoch 7 step 811: training loss: 33.86650816035693
Epoch 7 step 812: training accuarcy: 0.999
Epoch 7 step 812: training loss: 33.38997050923709
Epoch 7 step 813: training accuarcy: 0.9995
Epoch 7 step 813: training loss: 34.52110445125161
Epoch 7 step 814: training accuarcy: 0.9985
Epoch 7 step 814: training loss: 32.34357605857335
Epoch 7 step 815: training accuarcy: 0.9995
Epoch 7 step 815: training loss: 32.76080979603159
Ep

Epoch 7 step 892: training loss: 30.021005712313432
Epoch 7 step 893: training accuarcy: 0.9995
Epoch 7 step 893: training loss: 27.385529988427308
Epoch 7 step 894: training accuarcy: 1.0
Epoch 7 step 894: training loss: 26.444238024106994
Epoch 7 step 895: training accuarcy: 1.0
Epoch 7 step 895: training loss: 26.95295217740551
Epoch 7 step 896: training accuarcy: 1.0
Epoch 7 step 896: training loss: 27.959808265784304
Epoch 7 step 897: training accuarcy: 0.9995
Epoch 7 step 897: training loss: 29.21539195879513
Epoch 7 step 898: training accuarcy: 0.999
Epoch 7 step 898: training loss: 26.86335639864698
Epoch 7 step 899: training accuarcy: 1.0
Epoch 7 step 899: training loss: 27.607304828050303
Epoch 7 step 900: training accuarcy: 1.0
Epoch 7 step 900: training loss: 27.5211562433774
Epoch 7 step 901: training accuarcy: 1.0
Epoch 7 step 901: training loss: 27.91290352663156
Epoch 7 step 902: training accuarcy: 0.9995
Epoch 7 step 902: training loss: 27.913353322800937
Epoch 7 step 

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [06:33<00:00, 49.02s/it]


In [18]:
del fm_model
T.cuda.empty_cache()

### Train HRM FM Model

In [19]:
hrm_model = TorchHrmFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
hrm_model

TorchHrmFM()

In [20]:
adam_opt = optim.Adam(hrm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [21]:
hrm_learner = FMLearner(hrm_model, adam_opt, schedular, db)
hrm_learner

In [22]:
hrm_learner.fit(epoch=8,
                loss_callback=simple_loss_callback,
                log_dir=get_log_dir('kaggle', 'hrm'))

  0%|                                                                                                                                                                 | 0/8 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 49104.56763214461
Epoch 0 step 1: training accuarcy: 0.112
Epoch 0 step 1: training loss: 48457.58778517936
Epoch 0 step 2: training accuarcy: 0.10200000000000001
Epoch 0 step 2: training loss: 48962.610062743945
Epoch 0 step 3: training accuarcy: 0.1245
Epoch 0 step 3: training loss: 48548.96847246657
Epoch 0 step 4: training accuarcy: 0.1275
Epoch 0 step 4: training loss: 47459.58334606852
Epoch 0 step 5: training accuarcy: 0.1465
Epoch 0 step 5: training loss: 47724.11967629842
Epoch 0 step 6: training accuarcy: 0.14150000000000001
Epoch 0 step 6: training loss: 47846.84756590407
Epoch 0 step 7: training accuarcy: 0.137
Epoch 0 step 7: training loss: 47630.811309626995
Epoch 0 step 8: training accuarcy: 0.14300000000000002
Epoch 0 step 8: training loss: 46890.35215113851
Epoch 0 step 9: training accuarcy: 0.14200000000000002
Epoch 0 step 9: training loss: 46172.72804600619
Epoch 0 step 10: training accuarcy: 0.153
Epoch 0 step 10: training los

Epoch 0 step 86: training loss: 18909.942909548365
Epoch 0 step 87: training accuarcy: 0.5845
Epoch 0 step 87: training loss: 17364.675307133417
Epoch 0 step 88: training accuarcy: 0.6205
Epoch 0 step 88: training loss: 16654.727954710874
Epoch 0 step 89: training accuarcy: 0.634
Epoch 0 step 89: training loss: 16635.03591320687
Epoch 0 step 90: training accuarcy: 0.6365000000000001
Epoch 0 step 90: training loss: 14984.404132969783
Epoch 0 step 91: training accuarcy: 0.6715
Epoch 0 step 91: training loss: 14970.360199124605
Epoch 0 step 92: training accuarcy: 0.6755
Epoch 0 step 92: training loss: 13225.959184956548
Epoch 0 step 93: training accuarcy: 0.712
Epoch 0 step 93: training loss: 13763.68740450565
Epoch 0 step 94: training accuarcy: 0.6965
Epoch 0 step 94: training loss: 13164.94939376108
Epoch 0 step 95: training accuarcy: 0.7030000000000001
Epoch 0 step 95: training loss: 12441.060539406677
Epoch 0 step 96: training accuarcy: 0.726
Epoch 0 step 96: training loss: 11342.0840

 12%|███████████████████▏                                                                                                                                     | 1/8 [00:51<05:57, 51.03s/it]

Epoch: 1
Epoch 1 step 115: training loss: 7969.722969820458
Epoch 1 step 116: training accuarcy: 0.8275
Epoch 1 step 116: training loss: 8023.574539651519
Epoch 1 step 117: training accuarcy: 0.8245
Epoch 1 step 117: training loss: 7245.850289879148
Epoch 1 step 118: training accuarcy: 0.841
Epoch 1 step 118: training loss: 7009.452441708271
Epoch 1 step 119: training accuarcy: 0.8475
Epoch 1 step 119: training loss: 8074.418023554054
Epoch 1 step 120: training accuarcy: 0.8240000000000001
Epoch 1 step 120: training loss: 7842.338344856498
Epoch 1 step 121: training accuarcy: 0.8295
Epoch 1 step 121: training loss: 7402.593849453467
Epoch 1 step 122: training accuarcy: 0.8405
Epoch 1 step 122: training loss: 7173.281090398528
Epoch 1 step 123: training accuarcy: 0.843
Epoch 1 step 123: training loss: 6883.990589033709
Epoch 1 step 124: training accuarcy: 0.8485
Epoch 1 step 124: training loss: 7006.314368317555
Epoch 1 step 125: training accuarcy: 0.8495
Epoch 1 step 125: training loss

...

Epoch 6 step 775: training accuarcy: 0.9615
Epoch 6 step 775: training loss: 1603.5512894202554
Epoch 6 step 776: training accuarcy: 0.964
Epoch 6 step 776: training loss: 1549.1039703163337
Epoch 6 step 777: training accuarcy: 0.9665
Epoch 6 step 777: training loss: 1403.5415672458885
Epoch 6 step 778: training accuarcy: 0.969
Epoch 6 step 778: training loss: 1807.4142631067523
Epoch 6 step 779: training accuarcy: 0.962
Epoch 6 step 779: training loss: 1755.9015985737872
Epoch 6 step 780: training accuarcy: 0.9615
Epoch 6 step 780: training loss: 1670.3043249261586
Epoch 6 step 781: training accuarcy: 0.964
Epoch 6 step 781: training loss: 1623.015748922515
Epoch 6 step 782: training accuarcy: 0.966
Epoch 6 step 782: training loss: 1690.6778684346984
Epoch 6 step 783: training accuarcy: 0.964
Epoch 6 step 783: training loss: 1830.1994760210193
Epoch 6 step 784: training accuarcy: 0.9595
Epoch 6 step 784: training loss: 1564.8977937738841
Epoch 6 step 785: training accuarcy: 0.968
Epoc

 88%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                   | 7/8 [05:50<00:50, 50.04s/it]

Epoch: 7
Epoch 7 step 805: training loss: 2224.926208144229
Epoch 7 step 806: training accuarcy: 0.9505
Epoch 7 step 806: training loss: 1877.4336140917253
Epoch 7 step 807: training accuarcy: 0.9565
Epoch 7 step 807: training loss: 1954.6797397679984
Epoch 7 step 808: training accuarcy: 0.9565
Epoch 7 step 808: training loss: 1789.6648401751604
Epoch 7 step 809: training accuarcy: 0.9615
Epoch 7 step 809: training loss: 1871.9731062347973
Epoch 7 step 810: training accuarcy: 0.9595
Epoch 7 step 810: training loss: 1785.0710895667562
Epoch 7 step 811: training accuarcy: 0.96
Epoch 7 step 811: training loss: 1924.0741870615263
Epoch 7 step 812: training accuarcy: 0.9585
Epoch 7 step 812: training loss: 1753.7472427775908
Epoch 7 step 813: training accuarcy: 0.961
Epoch 7 step 813: training loss: 1715.5617916735328
Epoch 7 step 814: training accuarcy: 0.9625
Epoch 7 step 814: training loss: 1270.444419478824
Epoch 7 step 815: training accuarcy: 0.972
Epoch 7 step 815: training loss: 1769

Epoch 7 step 890: training accuarcy: 0.964
Epoch 7 step 890: training loss: 1763.4725245047885
Epoch 7 step 891: training accuarcy: 0.9615
Epoch 7 step 891: training loss: 1621.3485586164197
Epoch 7 step 892: training accuarcy: 0.963
Epoch 7 step 892: training loss: 1506.0314564787445
Epoch 7 step 893: training accuarcy: 0.967
Epoch 7 step 893: training loss: 1546.3678086656262
Epoch 7 step 894: training accuarcy: 0.968
Epoch 7 step 894: training loss: 1654.6104709844178
Epoch 7 step 895: training accuarcy: 0.9635
Epoch 7 step 895: training loss: 1577.813318090757
Epoch 7 step 896: training accuarcy: 0.9645
Epoch 7 step 896: training loss: 1647.068517417925
Epoch 7 step 897: training accuarcy: 0.963
Epoch 7 step 897: training loss: 1725.6682201180063
Epoch 7 step 898: training accuarcy: 0.963
Epoch 7 step 898: training loss: 1651.8529143363435
Epoch 7 step 899: training accuarcy: 0.964
Epoch 7 step 899: training loss: 1625.0471581363179
Epoch 7 step 900: training accuarcy: 0.9655
Epoch

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [06:41<00:00, 50.26s/it]


In [23]:
del hrm_model
T.cuda.empty_cache()

### Train PRME FM Model

In [24]:
prme_model = TorchPrmeFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
prme_model

TorchPrmeFM()

In [25]:
adam_opt = optim.Adam(prme_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [26]:
prme_learner = FMLearner(prme_model, adam_opt, schedular, db)
prme_learner

In [27]:
prme_learner.fit(epoch=8, loss_callback=simple_loss_callback, log_dir=get_log_dir('kaggle', 'prme'))

  0%|                                                                                                                                                                 | 0/8 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 49623.48128396946
Epoch 0 step 1: training accuarcy: 0.105
Epoch 0 step 1: training loss: 48873.34856646899
Epoch 0 step 2: training accuarcy: 0.1245
Epoch 0 step 2: training loss: 48772.25469438557
Epoch 0 step 3: training accuarcy: 0.136
Epoch 0 step 3: training loss: 48591.69769560142
Epoch 0 step 4: training accuarcy: 0.1385
Epoch 0 step 4: training loss: 48524.37249453151
Epoch 0 step 5: training accuarcy: 0.14
Epoch 0 step 5: training loss: 46739.499761136976
Epoch 0 step 6: training accuarcy: 0.1675
Epoch 0 step 6: training loss: 48115.74665151427
Epoch 0 step 7: training accuarcy: 0.1395
Epoch 0 step 7: training loss: 47197.856354077965
Epoch 0 step 8: training accuarcy: 0.1555
Epoch 0 step 8: training loss: 46640.16153866338
Epoch 0 step 9: training accuarcy: 0.1555
Epoch 0 step 9: training loss: 45342.490968963415
Epoch 0 step 10: training accuarcy: 0.17
Epoch 0 step 10: training loss: 46349.00888159786
Epoch 0 step 11: training accuarc

Epoch 0 step 86: training accuarcy: 0.181
Epoch 0 step 86: training loss: 37858.21036166166
Epoch 0 step 87: training accuarcy: 0.1825
Epoch 0 step 87: training loss: 37750.848116062116
Epoch 0 step 88: training accuarcy: 0.184
Epoch 0 step 88: training loss: 37668.99773688212
Epoch 0 step 89: training accuarcy: 0.1845
Epoch 0 step 89: training loss: 36749.47982047214
Epoch 0 step 90: training accuarcy: 0.20500000000000002
Epoch 0 step 90: training loss: 38048.29466173532
Epoch 0 step 91: training accuarcy: 0.17500000000000002
Epoch 0 step 91: training loss: 37478.72493630273
Epoch 0 step 92: training accuarcy: 0.1875
Epoch 0 step 92: training loss: 37673.83106061367
Epoch 0 step 93: training accuarcy: 0.183
Epoch 0 step 93: training loss: 38103.691997753005
Epoch 0 step 94: training accuarcy: 0.17300000000000001
Epoch 0 step 94: training loss: 37515.12181757482
Epoch 0 step 95: training accuarcy: 0.187
Epoch 0 step 95: training loss: 38546.88002983408
Epoch 0 step 96: training accuarc

 12%|███████████████████▏                                                                                                                                     | 1/8 [00:53<06:16, 53.73s/it]

Epoch: 1
Epoch 1 step 115: training loss: 34925.639471246526
Epoch 1 step 116: training accuarcy: 0.234
Epoch 1 step 116: training loss: 34041.26191688338
Epoch 1 step 117: training accuarcy: 0.251
Epoch 1 step 117: training loss: 33871.52605941463
Epoch 1 step 118: training accuarcy: 0.2505
Epoch 1 step 118: training loss: 32662.330786107646
Epoch 1 step 119: training accuarcy: 0.2775
Epoch 1 step 119: training loss: 31345.23160419582
Epoch 1 step 120: training accuarcy: 0.305
Epoch 1 step 120: training loss: 32140.35749981522
Epoch 1 step 121: training accuarcy: 0.296
Epoch 1 step 121: training loss: 30362.131924886664
Epoch 1 step 122: training accuarcy: 0.33
Epoch 1 step 122: training loss: 28139.78489340363
Epoch 1 step 123: training accuarcy: 0.38
Epoch 1 step 123: training loss: 27213.703842245042
Epoch 1 step 124: training accuarcy: 0.4025
Epoch 1 step 124: training loss: 27072.056154350335
Epoch 1 step 125: training accuarcy: 0.4045
Epoch 1 step 125: training loss: 27190.18588

...

Epoch 6 step 776: training loss: 6469.95161519717
Epoch 6 step 777: training accuarcy: 0.854
Epoch 6 step 777: training loss: 6351.394555798253
Epoch 6 step 778: training accuarcy: 0.8605
Epoch 6 step 778: training loss: 6599.207613894389
Epoch 6 step 779: training accuarcy: 0.8585
Epoch 6 step 779: training loss: 7156.851857184147
Epoch 6 step 780: training accuarcy: 0.8435
Epoch 6 step 780: training loss: 6219.492236798844
Epoch 6 step 781: training accuarcy: 0.8605
Epoch 6 step 781: training loss: 6077.428779418897
Epoch 6 step 782: training accuarcy: 0.868
Epoch 6 step 782: training loss: 6095.779268769322
Epoch 6 step 783: training accuarcy: 0.869
Epoch 6 step 783: training loss: 6352.14531418652
Epoch 6 step 784: training accuarcy: 0.8615
Epoch 6 step 784: training loss: 6533.739638352755
Epoch 6 step 785: training accuarcy: 0.857
Epoch 6 step 785: training loss: 6309.5478404581645
Epoch 6 step 786: training accuarcy: 0.861
Epoch 6 step 786: training loss: 6029.163394232861
Epoch

 88%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                   | 7/8 [06:05<00:52, 52.25s/it]

Epoch: 7
Epoch 7 step 805: training loss: 5837.822860693517
Epoch 7 step 806: training accuarcy: 0.8725
Epoch 7 step 806: training loss: 5872.779330019293
Epoch 7 step 807: training accuarcy: 0.872
Epoch 7 step 807: training loss: 6155.696513710112
Epoch 7 step 808: training accuarcy: 0.866
Epoch 7 step 808: training loss: 6186.359480544963
Epoch 7 step 809: training accuarcy: 0.865
Epoch 7 step 809: training loss: 6566.19268820615
Epoch 7 step 810: training accuarcy: 0.855
Epoch 7 step 810: training loss: 6582.039313615468
Epoch 7 step 811: training accuarcy: 0.858
Epoch 7 step 811: training loss: 5932.24287536907
Epoch 7 step 812: training accuarcy: 0.8715
Epoch 7 step 812: training loss: 5429.214777050631
Epoch 7 step 813: training accuarcy: 0.881
Epoch 7 step 813: training loss: 6130.576612448648
Epoch 7 step 814: training accuarcy: 0.865
Epoch 7 step 814: training loss: 6395.180063227235
Epoch 7 step 815: training accuarcy: 0.861
Epoch 7 step 815: training loss: 6585.61844887143
E

Epoch 7 step 889: training accuarcy: 0.8845000000000001
Epoch 7 step 889: training loss: 5906.359454037861
Epoch 7 step 890: training accuarcy: 0.871
Epoch 7 step 890: training loss: 5488.287287710072
Epoch 7 step 891: training accuarcy: 0.8805000000000001
Epoch 7 step 891: training loss: 5233.724942505251
Epoch 7 step 892: training accuarcy: 0.883
Epoch 7 step 892: training loss: 4602.219778548354
Epoch 7 step 893: training accuarcy: 0.9
Epoch 7 step 893: training loss: 4832.633448932175
Epoch 7 step 894: training accuarcy: 0.8955000000000001
Epoch 7 step 894: training loss: 5657.795660820183
Epoch 7 step 895: training accuarcy: 0.8765000000000001
Epoch 7 step 895: training loss: 4720.576335234174
Epoch 7 step 896: training accuarcy: 0.8945000000000001
Epoch 7 step 896: training loss: 5134.494897895243
Epoch 7 step 897: training accuarcy: 0.8865000000000001
Epoch 7 step 897: training loss: 5359.824407331974
Epoch 7 step 898: training accuarcy: 0.8835000000000001
Epoch 7 step 898: trai

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [06:57<00:00, 52.09s/it]


In [28]:
del prme_model
T.cuda.empty_cache()

### Train Trans FM Model

In [29]:
trans_model = TorchTransFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
trans_model

TorchTransFM()

In [30]:
adam_opt = optim.Adam(trans_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [31]:
trans_learner = FMLearner(trans_model, adam_opt, schedular, db)
trans_learner

In [32]:
trans_learner.fit(epoch=8,
                  loss_callback=trans_loss_callback,
                  log_dir=get_log_dir('kaggle', 'trans'))

  0%|                                                                                                                                                                 | 0/8 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 12160.892751540743
Epoch 0 step 1: training accuarcy: 0.9105
Epoch 0 step 1: training loss: 11826.89675858812
Epoch 0 step 2: training accuarcy: 0.927
Epoch 0 step 2: training loss: 11452.118953841327
Epoch 0 step 3: training accuarcy: 0.9470000000000001
Epoch 0 step 3: training loss: 11068.639538373965
Epoch 0 step 4: training accuarcy: 0.9550000000000001
Epoch 0 step 4: training loss: 10724.853203650406
Epoch 0 step 5: training accuarcy: 0.969
Epoch 0 step 5: training loss: 10366.000922362471
Epoch 0 step 6: training accuarcy: 0.971
Epoch 0 step 6: training loss: 10050.857014314888
Epoch 0 step 7: training accuarcy: 0.9695
Epoch 0 step 7: training loss: 9782.412504553706
Epoch 0 step 8: training accuarcy: 0.9605
Epoch 0 step 8: training loss: 9429.218339358878
Epoch 0 step 9: training accuarcy: 0.9695
Epoch 0 step 9: training loss: 9150.083329365123
Epoch 0 step 10: training accuarcy: 0.9655
Epoch 0 step 10: training loss: 8803.925872109134
Epo

Epoch 0 step 88: training loss: 419.4247955832205
Epoch 0 step 89: training accuarcy: 0.997
Epoch 0 step 89: training loss: 409.45225790489536
Epoch 0 step 90: training accuarcy: 0.994
Epoch 0 step 90: training loss: 384.0902134739546
Epoch 0 step 91: training accuarcy: 0.996
Epoch 0 step 91: training loss: 365.057570378091
Epoch 0 step 92: training accuarcy: 0.9965
Epoch 0 step 92: training loss: 349.4236759040039
Epoch 0 step 93: training accuarcy: 0.997
Epoch 0 step 93: training loss: 339.9839352012886
Epoch 0 step 94: training accuarcy: 0.9965
Epoch 0 step 94: training loss: 325.60015673319975
Epoch 0 step 95: training accuarcy: 0.995
Epoch 0 step 95: training loss: 320.0739945810385
Epoch 0 step 96: training accuarcy: 0.993
Epoch 0 step 96: training loss: 300.96693631516086
Epoch 0 step 97: training accuarcy: 0.995
Epoch 0 step 97: training loss: 283.46571088939487
Epoch 0 step 98: training accuarcy: 0.997
Epoch 0 step 98: training loss: 277.02326575982187
Epoch 0 step 99: trainin

 12%|███████████████████▏                                                                                                                                     | 1/8 [00:54<06:19, 54.15s/it]

Epoch: 1
Epoch 1 step 115: training loss: 134.75841000271495
Epoch 1 step 116: training accuarcy: 0.9955
Epoch 1 step 116: training loss: 120.88499411902873
Epoch 1 step 117: training accuarcy: 0.999
Epoch 1 step 117: training loss: 126.06403871832953
Epoch 1 step 118: training accuarcy: 0.9955
Epoch 1 step 118: training loss: 119.76052939137108
Epoch 1 step 119: training accuarcy: 0.996
Epoch 1 step 119: training loss: 110.92569184993953
Epoch 1 step 120: training accuarcy: 0.9985
Epoch 1 step 120: training loss: 108.791205074866
Epoch 1 step 121: training accuarcy: 0.997
Epoch 1 step 121: training loss: 102.83956020918073
Epoch 1 step 122: training accuarcy: 0.9965
Epoch 1 step 122: training loss: 103.65687674112475
Epoch 1 step 123: training accuarcy: 0.9955
Epoch 1 step 123: training loss: 100.94866732094351
Epoch 1 step 124: training accuarcy: 0.9965
Epoch 1 step 124: training loss: 96.67249741858993
Epoch 1 step 125: training accuarcy: 0.994
Epoch 1 step 125: training loss: 91.53

...

Epoch 6 step 776: training loss: 9.151883625292346
Epoch 6 step 777: training accuarcy: 0.999
Epoch 6 step 777: training loss: 11.358224378553803
Epoch 6 step 778: training accuarcy: 0.997
Epoch 6 step 778: training loss: 8.861355969446159
Epoch 6 step 779: training accuarcy: 0.998
Epoch 6 step 779: training loss: 11.73925168249439
Epoch 6 step 780: training accuarcy: 0.9975
Epoch 6 step 780: training loss: 16.24820744735153
Epoch 6 step 781: training accuarcy: 0.998
Epoch 6 step 781: training loss: 10.629463208345193
Epoch 6 step 782: training accuarcy: 0.9975
Epoch 6 step 782: training loss: 14.186852483782042
Epoch 6 step 783: training accuarcy: 0.995
Epoch 6 step 783: training loss: 12.68431564714567
Epoch 6 step 784: training accuarcy: 0.9975
Epoch 6 step 784: training loss: 10.347415631869952
Epoch 6 step 785: training accuarcy: 0.998
Epoch 6 step 785: training loss: 7.437622268654307
Epoch 6 step 786: training accuarcy: 0.9995
Epoch 6 step 786: training loss: 17.04763048066891
E

 88%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                   | 7/8 [06:20<00:54, 54.47s/it]

Epoch: 7
Epoch 7 step 805: training loss: 7.11458540766523
Epoch 7 step 806: training accuarcy: 0.999
Epoch 7 step 806: training loss: 13.44953423471516
Epoch 7 step 807: training accuarcy: 0.9975
Epoch 7 step 807: training loss: 13.380260211985263
Epoch 7 step 808: training accuarcy: 0.9955
Epoch 7 step 808: training loss: 6.594813322440129
Epoch 7 step 809: training accuarcy: 0.9995
Epoch 7 step 809: training loss: 10.091206933802434
Epoch 7 step 810: training accuarcy: 0.998
Epoch 7 step 810: training loss: 10.236788300835059
Epoch 7 step 811: training accuarcy: 0.9975
Epoch 7 step 811: training loss: 9.90358998516715
Epoch 7 step 812: training accuarcy: 0.999
Epoch 7 step 812: training loss: 14.426919063718644
Epoch 7 step 813: training accuarcy: 0.9965
Epoch 7 step 813: training loss: 7.373284149902315
Epoch 7 step 814: training accuarcy: 0.999
Epoch 7 step 814: training loss: 8.71742184036349
Epoch 7 step 815: training accuarcy: 0.9985
Epoch 7 step 815: training loss: 5.483928684

Epoch 7 step 891: training loss: 12.622982812292847
Epoch 7 step 892: training accuarcy: 0.997
Epoch 7 step 892: training loss: 13.019636853345306
Epoch 7 step 893: training accuarcy: 0.997
Epoch 7 step 893: training loss: 7.6545537319614825
Epoch 7 step 894: training accuarcy: 0.9975
Epoch 7 step 894: training loss: 6.861583488200544
Epoch 7 step 895: training accuarcy: 0.999
Epoch 7 step 895: training loss: 9.571752318098444
Epoch 7 step 896: training accuarcy: 0.9985
Epoch 7 step 896: training loss: 15.39481397733446
Epoch 7 step 897: training accuarcy: 0.995
Epoch 7 step 897: training loss: 10.93101306955965
Epoch 7 step 898: training accuarcy: 0.998
Epoch 7 step 898: training loss: 6.983164075279247
Epoch 7 step 899: training accuarcy: 0.999
Epoch 7 step 899: training loss: 10.972670067328423
Epoch 7 step 900: training accuarcy: 0.9975
Epoch 7 step 900: training loss: 7.1518080557438175
Epoch 7 step 901: training accuarcy: 0.9985
Epoch 7 step 901: training loss: 5.350563701460016


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [07:15<00:00, 54.35s/it]
